In [42]:
options(repr.plot.width=5, repr.plot.height=4, scipen = 999)

In [43]:
list.of.packages <- c('ggplot2', 'R.matlab', 'ggthemes')

new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) install.packages(new.packages, repos = "https://cran.r-project.org")

library(ggplot2)
library(R.matlab)
library(ggthemes)

In [44]:
df.train <- read.csv("train.csv")

In [45]:
X <- as.matrix(df.train[, 2:ncol(df.train)])
y <- df.train$label
y[y == 0] <- 10

dim(X)

[1] 42000   784

In [46]:
X_norm <- X / 255
str(X_norm)
dim(X_norm)

 num [1:42000, 1:784] 0 0 0 0 0 0 0 0 0 0 ...
 - attr(*, "dimnames")=List of 2
  ..$ : NULL
  ..$ : chr [1:784] "pixel0" "pixel1" "pixel2" "pixel3" ...


[1] 42000   784

In [47]:
pca <- function(X) {
    
    m <- nrow(X)
    n <- ncol(X)
    
    U <- rep(0, n)
    S <- rep(0, n)
    
    #Calculamos la matriz de covarianza
    Sigma <- (1 / m) * (t(X) %*% X) # matriz nxn
    
    USV <- svd(Sigma)
    
    return(USV)
}

USV <- pca(X_norm)

#U contiene el componente principal, S($d en este caso) contiene la matriz diagonal
S <- USV$d
U <- USV$u

In [48]:
K <- 100

projectData <- function(X, U, K) {
    
    
    Z <- matrix(0, nrow = nrow(X), ncol = K)
    U_reduce <- U[,1:K]
    
    Z <- X %*% U_reduce #o t(U_reduce) %*% X    si X es una matriz y no un dataframe
    
    Z    
}

Z <- projectData(X_norm, U, K)
str(Z) #Los datos proyectados ahora tienen una dimensión de 5000 x 100

 num [1:42000, 1:400] -3.69 -8.7 -3.64 -2.77 -9.13 ...


In [49]:
X <- Z

In [50]:
#Creamos la función sigmoidal
sigmoid <- function(z) {
  
  g <- 1 / (1 + exp(1) ^ (-z))
}

#Función para calcular el gradiente de la función sigmoidal
sigmoidGradient <- function(z) {
    
    z <- sigmoid(z)
    g  <- z * (1 - z)
    g
}

In [51]:
#Funcion para calcular el costo (con regularización de parámetros)

nnCostFunction <- function(nn_params) {
    
    Theta1 <- matrix(nn_params[1:(hidden_layer_size *(input_layer_size + 1))], hidden_layer_size, input_layer_size +1)
    Theta2 <- matrix(nn_params[(1 + hidden_layer_size * (input_layer_size + 1)):length(nn_params)], 
                     num_labels, hidden_layer_size +1)
    
    Theta1_nz <- Theta1[, 2:ncol(Theta1)]
    Theta2_nz <- Theta2[, 2:ncol(Theta2)]
    
    m <- nrow(X)
    J <- 0
    
    X <- cbind(1, X)
    a1 <- X
    z2 <- a1 %*% t(Theta1)
    a2 <- sigmoid(z2)
    a2 <- cbind(1, a2)
    z3 <- a2 %*% t(Theta2)
    a3 <- sigmoid(z3)
    
    hyp <- a3
    y_mat <- diag(num_labels)
    y_mat <- y_mat[, y]
    
    inner_value <- - t(y_mat) * log(hyp) - (1 - t(y_mat)) * log(1 - hyp)
    J_noreg <- (1/m) * sum(sum(inner_value))
    reg_term <- (lambda / (2 * m)) * sum(sum(sum(Theta1_nz ^ 2)) + sum(sum(Theta2_nz ^ 2)))
    
    J <- J_noreg + reg_term
    
    J
    
}

In [52]:
#Función para calcular los gradientes (con regularización de parámetros)
nnGradFunction <- function(nn_params) {
    
    Theta1 <- matrix(nn_params[1:(hidden_layer_size *(input_layer_size + 1))], hidden_layer_size, input_layer_size +1)
    Theta2 <- matrix(nn_params[(1 + hidden_layer_size * (input_layer_size + 1)):length(nn_params)], 
                     num_labels, hidden_layer_size +1)
    
    Theta1_grad <- matrix(0, ncol = ncol(Theta1), nrow = nrow(Theta1))
    Theta2_grad <- matrix(0, ncol = ncol(Theta1), nrow = nrow(Theta2))
    
    Theta1_nz <- Theta1[, 2:ncol(Theta1)]
    Theta2_nz <- Theta2[, 2:ncol(Theta2)]
    
    m <- nrow(X)
   
    X <- cbind(1, X)
    a1 <- X
    z2 <- a1 %*% t(Theta1)
    a2 <- sigmoid(z2)
    a2 <- cbind(1, a2)
    z3 <- a2 %*% t(Theta2)
    a3 <- sigmoid(z3)
    
    hyp <- a3
    y_mat <- diag(num_labels)
    y_mat <- y_mat[, y]
    
    delta3 <- a3 - t(y_mat)
    z2 <- cbind(1, z2)
    delta2 <- (delta3 %*% Theta2) * sigmoidGradient(z2)
    delta2 <- delta2[, 2:ncol(delta2)]
    
    cap_delta1 <- 0
    cap_delta2 <- 0
    
    cap_delta1 <- cap_delta1 + t(delta2) %*% a1
    cap_delta2 <- cap_delta2 + t(delta3) %*% a2
    
    Theta1[, 1] <- 0
    Theta2[, 1] <- 0
    
    Theta1_grad <- (1 / m) * (cap_delta1 + lambda * Theta1)
    Theta2_grad <- (1 / m) * (cap_delta2 + lambda * Theta2)
    
    grad <- c(c(Theta1_grad), c(Theta2_grad))
}

In [53]:
#Entrenamiento de la red neuronal----------------------------------------------------

#Inicializamos los parámetros de manera aleatoria
randInitializeWeights <- function(L_in, L_out) {
    
    W <- matrix(0, L_out, 1 + L_in)
    epsilon_init <- 0.12
  
    rnd <- runif(L_out * (1 + L_in))
    rnd <- matrix(rnd,L_out,1 + L_in)
    W <- rnd * 2 * epsilon_init - epsilon_init
    W
}

n <- ncol(X)
m <- nrow(X)

input_layer_size <- n
hidden_layer_size <- 25
num_labels <- 10
lambda <- 1

In [54]:
initial_Theta1 <- matrix(runif((n+1)*hidden_layer_size), nrow = hidden_layer_size, ncol=n + 1)
initial_Theta2 <- matrix(runif(num_labels * (hidden_layer_size + 1)), nrow = num_labels, ncol = hidden_layer_size + 1)

initial_nn_params <- c(c(initial_Theta1), c(initial_Theta2))

In [55]:
results <- optim(initial_nn_params, fn = nnCostFunction, gr = nnGradFunction, method = "BFGS", 
               control = list(maxit=1000, trace=1, REPORT=1) )

theta <- results$par
J <- results$value

cat("Costo final:", J)

initial  value 40.931144 
iter   2 value 6.430348
iter   3 value 5.163576
iter   4 value 3.447467
iter   5 value 3.306275
iter   6 value 3.277189
iter   7 value 3.272924
iter   8 value 3.265736
iter   9 value 3.239598
iter  10 value 3.221371
iter  11 value 3.167757
iter  12 value 3.138721
iter  13 value 3.126424
iter  14 value 3.004254
iter  15 value 2.967723
iter  16 value 2.946442
iter  17 value 2.935973
iter  18 value 2.925414
iter  19 value 2.904745
iter  20 value 2.884090
iter  21 value 2.870234
iter  22 value 2.848562
iter  23 value 2.824281
iter  24 value 2.789304
iter  25 value 2.741940
iter  26 value 2.692921
iter  27 value 2.603831
iter  28 value 2.555139
iter  29 value 2.534755
iter  30 value 2.427651
iter  31 value 2.371547
iter  32 value 2.261448
iter  33 value 2.232102
iter  34 value 2.195951
iter  35 value 2.147459
iter  36 value 2.088956
iter  37 value 2.028132
iter  38 value 1.971853
iter  39 value 1.917112
iter  40 value 1.883461
iter  41 value 1.867106
iter  42 value

In [56]:
length(initial_nn_params)
length(theta)

[1] 10285

[1] 10285

In [57]:
Theta1 <- matrix(theta[1:(hidden_layer_size *(input_layer_size + 1))], hidden_layer_size, input_layer_size +1)
Theta2 <- matrix(theta[(1 + hidden_layer_size * (input_layer_size + 1)):length(initial_nn_params)], 
                     num_labels, hidden_layer_size +1)

In [58]:
df.test <- read.csv("test.csv")
X <- as.matrix(df.test[, 2:ncol(df.test)])
X <- cbind(0, X)
X_norm <- X / 255

In [59]:
#Pca al dataset de test

In [60]:
USV <- pca(X_norm)

#U contiene el componente principal, S($d en este caso) contiene la matriz diagonal
S <- USV$d
U <- USV$u

K <- 400

Z_test <- projectData(X_norm, U, K)

In [61]:
dim(Z_test)

[1] 28000   400

In [62]:
#Calculamos la precisión de la red

predict <- function(Theta1, Theta2, X) {
    
    m <- nrow(X)
    num_labels <- nrow(Theta2)
    
    p <- rep(0, m)
    
    h1 <- sigmoid(cbind(1, X) %*% t(Theta1))
    h2 <- sigmoid(cbind(1, h1) %*% t(Theta2))
    
    p <- apply(h2, 1, which.max)
    p
}

In [63]:
predictions <- predict(Theta1, Theta2, Z_test)

predictions[predictions == 10] <- 0

m <- nrow(df.test)
ids <- 1:m

In [64]:
submit <- data.frame(ImageId = ids, Label = predictions)

In [65]:
write.csv(submit, file = "cuartosubmit.csv", row.names = FALSE, quote = FALSE)